<a href="https://colab.research.google.com/github/FireStrings/MasterDegree/blob/main/CNN_LSTM/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import os
os.chdir("/content/drive/My Drive/UFSCAR/Notebooks/LSTM")

Mounted at /content/drive/


###Imports

In [ ]:
%run LSTM.ipynb

Mounted at /content/drive/


In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, Flatten, GlobalAveragePooling1D, Concatenate, MaxPooling1D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential

###Functions

In [ ]:
def create_cnn_model(janela_tempo, n_features):
    input_clima = Input(shape=(janela_tempo, n_features), name="input_clima")

    x_cnn = Conv1D(filters=64, kernel_size=3, activation='relu', name="cnn_layer_1")(input_clima)
    x_cnn = BatchNormalization(name="batch_norm_layer_1")(x_cnn)
    x_cnn = Conv1D(filters=32, kernel_size=3, activation='relu', name="cnn_layer_2")(x_cnn)
    x_cnn = MaxPooling1D(name="max_pool_layer_1")(x_cnn)
    x_cnn = Flatten()(x_cnn)
    x_cnn = Dense(32, activation='relu')(x_cnn)
    x_cnn = BatchNormalization()(x_cnn)

    return x_cnn, input_clima

def create_cnn_lstm_model(janela_tempo, n_features, with_compiled, with_bilstm):

    x_cnn, input_clima = create_cnn_model(janela_tempo, n_features)
    x_lstm, input_rad = create_lstm_model(janela_tempo, "input_radiacao", with_compiled, with_bilstm)

    x = Concatenate()([x_cnn, x_lstm])
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1, name="saida", activation='linear')(x)

    model = Model(inputs=[input_clima, input_rad], outputs=output)
    model.compile(optimizer='adam', loss='mse')

    return model


def training_model(modelo, Xc_train, Xr_train, y_train, Xc_val, Xr_val, y_val):
    modelo.fit(
        x={"input_clima": Xc_train, "input_radiacao": Xr_train},
        y=y_train,
        validation_data=(
            {"input_clima": Xc_val, "input_radiacao": Xr_val},
            y_val
        ),
        epochs=100,
        batch_size=32,
        verbose=0,
        # shuffle=False
    )

def process_cnn_lstm(times, station, anual_df, results, with_plot, with_bilstm):
    nn_type = "BiLSTM" if with_bilstm else "LSTM"
    for i in range(times):
        print("Starting process " + str(i) + " for station " + station + " with " + nn_type + " type")
        results[station][i] = {"train_pred":None, "test_pred": None, "period": None, "metrics":None}

        # list_cols = ["pto_orvalho_min_c", "umi_min_perc", "pressao_min_hpa", "radiacao"]
        list_cols = ["temp_ins_c", "pressao_max_hpa", "vel_vento_ms", "radiacao"] #new cols

        df_norm = select_and_normalize_divided(anual_df, list_cols)

        # df_inupt = df_norm
        df_inupt = df_norm[["temp_ins_c", "pressao_max_hpa", "vel_vento_ms"]] #TESTE
        df_target = df_norm[["radiacao"]]

        X_clima, X_rad, y = data_to_input_and_output_cnn_lstm(df_inupt, df_target, janela=janela_tempo, horizonte=1)
        Xc_train, Xc_val, Xr_train, Xr_val, y_train, y_val = train_test_split(X_clima, X_rad, y, test_size=0.3, random_state=42, shuffle=False)

        model = create_cnn_lstm_model(janela_tempo, df_inupt.shape[1], False, with_bilstm)
        training_model(model, Xc_train, Xr_train, y_train, Xc_val, Xr_val, y_val)

        train_pred = model.predict(x={"input_clima": Xc_train, "input_radiacao": Xr_train}, verbose=0)
        test_pred = model.predict(x={"input_clima": Xc_val, "input_radiacao": Xr_val}, verbose=0)

        if with_plot:
            period = anual_df[train_pred.shape[0]:]['data'].iloc[:test_pred.shape[0]]
            plot_results_2(test_pred.reshape(test_pred.shape[0]), y_val, period)

        results[station][i]["train_pred"] = train_pred
        results[station][i]["test_pred"] = test_pred
        results[station][i]["metrics"] = get_metrics(test_pred, y_val)
